In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [53]:
class DadosAbertos:
    
    def __init__(self, url_conjunto, extensoes=('csv',)):
        
        self.url_conjunto = url_conjunto
        self.extensoes=extensoes
        self.recursos = self.parse_all_recursos(url_conjunto, extensoes)
    
    def get_page_conjunto(self, url):
        
        with requests.get(url) as r:
            html = r.text
        
        return html
    
    def gerar_sopa(self, html):
        
        return BeautifulSoup(html)
    
    def listar_recursos(self, sopa):
        
        return sopa.find_all('li', {'class' : "resource-item"})
    
    def pegar_extensao(self, link):
        
        return link.split('.')[-1]
    
    def parsear_recurso(self, recurso):
        
        desc = recurso.find('p', {'class' : 'description'}).text.strip()
        link = recurso.find('a', {'class' : "resource-url-analytics"})['href']
        extensao = self.pegar_extensao(link)
        
        dados = {
            'descricao' : desc,
            'extensao' : extensao,
            'link' : link
        } 
        
        return dados
    
    def check_extensao_suportada(self, extensoes_solicitadas):
        
        extensoes_suportadas = ('csv', 'xlsx', 'xls', 'txt')
        for extensao in extensoes_solicitadas:
            if extensao not in extensoes_suportadas:
                raise NotImplementedError(f'Extensao {extensao} não suportada')
    
    def parse_all_recursos(self, url_conjunto, extensoes=('csv', )):
        
        self.check_extensao_suportada(extensoes)
        
        page_conjunto = self.get_page_conjunto(url_conjunto)
        sopa = self.gerar_sopa(page_conjunto)
        
        parsed_data = []
        recursos_raw = self.listar_recursos(sopa)
        
        for rec in recursos_raw:
            
            parsed_resource = self.parsear_recurso(rec)
            
            if parsed_resource['extensao'] in extensoes:
            
                parsed_data.append(parsed_resource)
            
            else:
                print('Resource não listado por estar fora da extensão solicitada:')
                print(f"{parsed_resource['descricao']} : {parsed_resource['extensao']}")
        
        return parsed_data

                
    def check_if_text_download(self, recurso_parsed):
        
        extensao = recurso_parsed['extensao']
        self.check_extensao_suportada((extensao,))
        
        extensoes_textuais = ('csv', 'txt', 'xls', 'xlsx')
        
        if extensao in extensoes_textuais:
            return True
        return False
    
    def get_content(self, recurso_parsed):
        
        text_download = self.check_if_text_download(recurso_parsed)
        
        if text_download:
            with requests.get(recurso_parsed['link']) as r:
                return r.text
        else:
            with requests.get(recurso_parsed['link']) as r:
                    return r.content
    
    def download_all(self, resource_list):
        
        for resource in resource_list:
            yield self.get_content(resource)
            
    def __call__(self):
        
        return self.download_all(self.recursos)

In [54]:
url_cadastro_escolas = ('http://dados.prefeitura.sp.gov.br/pt_PT/'
                        'dataset/cadastro-de-escolas-municipais-conveniadas-e-privadas')

In [62]:
data[1][:2000]

'ï»¿dre;codesc;tipoesc;nomesc;ceu;diretoria;subpref;endereco;numero;bairro;cep;tel1;tel2;fax;situacao;coddist;distrito;setor;codinep;cd_cie;eh;fx_etaria;dt_criacao;ato_criacao;dom_criacao;dt_ini_conv;dt_ini_func;dt_autoriza;dt_extintao;nome_ant;rede;latitude;longitude;database\r\nBT;191;EMEF;ALIPIO CORREA NETO, PROF;;DIRETORIA REGIONAL DE EDUCACAO BUTANTA;BUTANTA;Avenida JOAO CAIAFFA;140;JARDIM TABOAO;5742100;58450121;;;ATIVA;94;VILA SONIA;9404;35098462.0;38488;1,621E+14;;27/05/1988;26.003;27/05/1988;;02/10/1988;16/03/1991;;ALIPIO CORREA NETO, PROF.;DIR;-23.612.237;-46.749.888;01/12/2019\r\nBT;213;EMEI;ANTONIO CARLOS PACHECO E SILVA, PROF;;DIRETORIA REGIONAL DE EDUCACAO BUTANTA;BUTANTA;Avenida JOÃ\x83O CAIAFFA;90;JARDIM TABOÃ\x83O;5742100;58448902;;;ATIVA;94;VILA SONIA;9404;35098504.0;13506;1,621E+14;;07/06/1988;26.117;07/06/1988;;26/09/1988;13/03/2001;;ANTONIO CARLOS PACHECO E SILVA, PROF.;DIR;-23.611.929;-46.750.176;01/12/2019\r\nBT;477;EMEF;PROFA. EDA TEREZINHA CHICA MEDEIROS;;DIRET